In [ ]:
import sys, os
sys.path.append(os.path.abspath(os.path.join('..')))
from Code.logistic_regression import log_loss_ridge, import_breast_cancer, accuracy_func, logistic_loss_func, logistic_grad, ridge_term
from Code.utilities import train_test_split, plot_test_results
from Code.neural_network import _beta_init, neural_network_model
from Code.descent_methods import SGD_adam

from matplotlib import pyplot as plt

import jax.numpy as jnp
import numpy as np
from jax import nn, grad

In [ ]:
X, y = import_breast_cancer()

In [ ]:
def single_run(X, y, lr, lam, hidden_layer_num, node_per_hidden, hidden_activation, test_index=None):
    layer_list = [X.shape[1]] + [node_per_hidden]*hidden_layer_num + [1]
    beta0 = _beta_init(layer_list)

    batch_size = 50
    epochs = 15

    model = lambda beta, X: neural_network_model(beta, X, activation=hidden_activation, output_activation=nn.sigmoid)

    loss_func = logistic_loss_func(model=model)


    #loss_func = log_loss_ridge(model=model, lam=lam)

    # loss_grad = grad(loss_func)
    loss_grad = grad(loss_func)
    print(loss_grad(beta0, X, y))

    test_func = accuracy_func(model)

    X_train, y_train, X_test, y_test, test_index = train_test_split(X, y, 0.2, test_index=test_index)

    result = SGD_adam(X_train, y_train, X_test, y_test, grad_method=loss_grad,beta0=beta0, n_epochs=epochs, batch_size=batch_size, test_loss_func=loss_func, lr=lr)

    m = int(len(y_train) / batch_size)
    result.update({"test_index":test_index, "m":m})

    plot_test_results(result["test_loss_list"], result["train_loss_list"], result["m"])
    # result2 = SGD_adam(X_train, y_train, X_test, y_test, grad_method=loss_grad,beta0=beta0, n_epochs=epochs, batch_size=batch_size, test_loss_func=test_func, lr=lr)
    # plot_test_results(result2["test_loss_list"], result2["train_loss_list"], result["m"])


    return result




print(y.shape)
print(X.shape)
result = single_run(X, y, 0.01, 0.001, hidden_layer_num=2, node_per_hidden=5, hidden_activation=nn.leaky_relu)
print("Log run")
#result = single_run(X, y, 0.01, 0.001, 0, 5, hidden_activation=nn.relu)



In [ ]:
def search_lam():
    lams = np.logspace(-8, 2, num=30)
    test_errs = []
    train_errs = []
    for lam in lams:
        print(lam)
        result = single_run(X, y, lr=0.01, lam=lam, hidden_layer_num=2, node_per_hidden=5, hidden_activation=nn.leaky_relu)
        test_errs.append(result["test_loss_list"][-1])
        train_errs.append(result["train_loss_list"][-1])

    plt.plot(lams, test_errs)
    plt.plot(lams, train_errs)
    plt.xscale("log")
    plt.show()

search_lam()